In [ ]:
#imported the needed packagese
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from sklearn.model_selection import cross_val_score
from wordcloud import WordCloud
import nltk
from imblearn.over_sampling import SMOTE
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv("22205499.csv")
df.head()

In [ ]:
import pandas as pd

# Select the desired columns
df = df[['category', 'headline', 'authors', 'link', 'short_description', 'date']]
print(df)

In [ ]:
#THIS IS CLEARLY A UNBALANED DATA
df["category"].value_counts()  

In [ ]:
#made the category columns into binary as it can make easy for converting while making model
df['id'] = (df['category'] == 'WORLDPOST').astype(int)
selected_columns = ['category', 'headline', 'authors', 'link', 'short_description', 'date']
df.head()

In [ ]:
#We only need two columns for our work. so we will look for null values in that only
print("Total null values in headline column " + str(df['headline'].isnull().sum()))
print()
print("Total null values in category column " + str(df['category'].isnull().sum()))

In [ ]:
# Check for blank values in all the columns
print("Blank Values:")
print(df.isnull().sum())

In [ ]:
df.shape

In [ ]:
#We will need to drop the null records on reduce the noise in the dataset
df= df.dropna(subset=['headline', 'category']).copy()
df.head()

In [ ]:
import pandas as pd


# Calculate the length of each headline in the "headline" column and named it in new colummn "headline_length":
df['headline_length'] = df['headline'].str.len()

#Calcualted the statiscs of the model after removing the null values
mean_headline_len = df['headline_length'].mean()
std_headline_len = df['headline_length'].std()
max_headline_len = df['headline_length'].max()
min_headline_len = df['headline_length'].min()

# Define the outlier range as two standard deviations above and below the mean headline length:
upper_bound = mean_headline_len + 2 * std_headline_len
lower_bound = mean_headline_len - 2 * std_headline_len


#To find the outliers
outliers = df[(df['headline_length'] > upper_bound) | (df['headline_length'] < lower_bound)]

print("Mean headline_length:", mean_headline_len)
print("Standard deviation headline_length:", std_headline_len)
print("Maximum headline_length:", max_headline_len)
print("Minimum headline_length:", min_headline_len)
print("Number of outliers in 'headline_length' column:", len(outliers))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#the length of the text_length column
df['headline_length'] = df['headline'].str.len()

# Create the box plot for the "text_length" column
plt.boxplot(df['headline_length'])
plt.xlabel('Text Length')
plt.ylabel('Length')
plt.title('Box Plot of Length of columns:')
plt.show()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')


# Filter the DataFrame to include only "FOOD & DRINK" and "WORLDPOST" categories
food_drink_df = df[df['category'] == 'FOOD & DRINK']
worldpost_df = df[df['category'] == 'WORLDPOST']

# Drop rows with missing values in the "headline" column for both categories
food_drink_df = food_drink_df.dropna(subset=['headline'])
worldpost_df = worldpost_df.dropna(subset=['headline'])

# Initialize the CountVectorizer with stop_words='english'
vectorizer = CountVectorizer(stop_words='english')

#converted the FOOD & DRINK category into a word count matrix
food_drink_words = vectorizer.fit_transform(food_drink_df['headline'])

#converted the WORLDPOST category into a word count matrix
worldpost_words = vectorizer.fit_transform(worldpost_df['headline'])

# Create a DataFrame with words and their counts for FOOD & DRINK category, sorted in descending order
food_drink_word_count = zip(vectorizer.get_feature_names_out(), np.asarray(food_drink_words.sum(axis=0)).ravel())
df_food_drink_words = pd.DataFrame(food_drink_word_count, columns=['Word', 'Count'])
df_food_drink_words.sort_values("Count", ascending=False, inplace=True)

#DataFrame with words and their counts for "WORLDPOST" category, sorted in descending order of count
worldpost_word_count = zip(vectorizer.get_feature_names_out(), np.asarray(worldpost_words.sum(axis=0)).ravel())
df_worldpost_words = pd.DataFrame(worldpost_word_count, columns=['Word', 'Count'])
df_worldpost_words.sort_values("Count", ascending=False, inplace=True)

#most common words in the "FOOD & DRINK" category
print("Most common words in 'FOOD & DRINK' category:")
print(df_food_drink_words.head(10))

#most common words in the "WORLDPOST" category
print("Most common words in 'WORLDPOST' category:")

print(df_worldpost_words.head(10))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#creatd dataFrame to include only the "FOOD & DRINK" category
food_drink_df = df[df['category'] == 'FOOD & DRINK']

# dropped rows with missing values in the "headline" column
food_drink_df = food_drink_df.dropna(subset=['headline'])

# Initialize the CountVectorizer with stop_words='english'
vectorizer = CountVectorizer(stop_words='english')

# converted the headline in the "FOOD & DRINK" category into a word count.
words_in_food_drink_category = vectorizer.fit_transform(food_drink_df['headline'])

# Create a DataFrame with words and their counts, sorted in descending order
word_count = zip(vectorizer.get_feature_names_out(), np.asarray(words_in_food_drink_category.sum(axis=0)).ravel())
df_words = pd.DataFrame(word_count, columns=['Word', 'Count'])
df_words.sort_values("Count", ascending=False, inplace=True)

# Display the most used words in the "FOOD & DRINK" category
most_used_words = df_words.head(10)

# Display the least used words in the "FOOD & DRINK" category
least_used_words = df_words.tail(10)

# Extract the most used and least used words from the DataFrame and save them to lists
words_most = most_used_words['Word'].tolist()
most_word_counts_list = most_used_words['Count'].tolist()

words_least = least_used_words['Word'].tolist()
least_word_counts_list = least_used_words['Count'].tolist()

# Create a dictionary of word frequencies
word_frequency_most = dict(zip(words_most, most_word_counts_list))
word_frequency_least = dict(zip(words_least, least_word_counts_list))

# Create a word cloud object with my parameters
wordcloud_most = WordCloud(width=500, height=500, background_color='green').generate_from_frequencies(word_frequency_most)
wordcloud_least = WordCloud(width=400, height=400, background_color='red').generate_from_frequencies(word_frequency_least)

# Display the most used word cloud using matplotlib
plt.figure(figsize=(8, 4))
plt.imshow(wordcloud_most, interpolation='bilinear')
plt.axis('off')
plt.title("Most Used Words in FOOD & DRINK Category", fontsize=13)
plt.show()

# Display the least used word cloud using matplotlib
plt.figure(figsize=(8, 4))
plt.imshow(wordcloud_least, interpolation='bilinear')
plt.axis('off')
plt.title("Least Used Words in WORLDPOST Category", fontsize=13)
plt.show()

print("Most times", most_used_words)
print()
print("Least times", least_used_words)

#We can see how many words are their with accruacy based on their size

# Data Preparation & Modelling

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

#made the headlnie column into lowercase to remove any kind of confusion for same word
df['headline'] = df['headline'].str.lower()

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df['headline'] = df['headline'].apply(remove_stopwords)
print(df.head(2))


In [ ]:
#Here we will assign the "data" and "target" values 
X = df['headline']
y = df['category']

In [ ]:
#Next we will split the dataset

#Here we will convert the labels to represent it numerically
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, random_state=0, test_size = 0.30, train_size = 0.7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=0, test_size = 0.199/0.7, train_size = 0.5/0.7,stratify=y_train_valid)


#Lets look at the size of each set.
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

Explanation:
The code above will first perform an initial train-validation-test split using the train_test_split function(30% for test and 70% for train).
Then it will split the remaining data from the previous split into a train set (X_train, y_train) and a validation set (X_valid, y_valid) ((0.199/0.7) of the remaining data the validation set and 0.5/0.7 for test set)
The random_state parameter is used for reproducibility, while stratify parameter ensures the class distribution will be preserved.


In [ ]:
#We will convert the dataframe into a csv.

#First we will convert them to a dataframe
train_df = pd.DataFrame({'headline': X_train, 'category': y_train})
test_df = pd.DataFrame({'headline': X_test, 'category': y_test})
valid_df = pd.DataFrame({'headline': X_valid, 'category': y_valid})

#Next the train, test, and validation sets will be saved as CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)
valid_df.to_csv('valid.csv', index=False)

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize


# Load the train and valid dataset into dataframes
train_df = pd.read_csv("train.csv")
valid_df = pd.read_csv("valid.csv")

# Define a function to tokenize the text data using NLTK's word_tokenize function
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return []

# Tokenize the text  in both train and valid dataset
train_df['processed_text'] = train_df['headline'].apply(tokenize_text)
valid_df['processed_text'] = valid_df['headline'].apply(tokenize_text)

print(train_df.head(11))

In [ ]:
# conveted the data intor numerical data using TF-IDF vectorization
tfidf = TfidfVectorizer()
training_vectors = tfidf.fit_transform(train_df['processed_text'].apply(' '.join))
valid_vectors = tfidf.transform(valid_df['processed_text'].apply(' '.join))

In [ ]:
# Load the training and validation datasets into dataframes
train_df = pd.read_csv("train.csv")
valid_df = pd.read_csv("valid.csv")

# Define a function to tokenize the text data
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return []

# Tokenize the text data in both train and valid dataset

train_df['processed_text'] = train_df['headline'].apply(tokenize_text)
valid_df['processed_text'] = valid_df['headline'].apply(tokenize_text)

# conveted the data intor numerical data using TF-IDF vectorization
tfidf = TfidfVectorizer()
training_vectors = tfidf.fit_transform(train_df['processed_text'].apply(' '.join))
validation_vectors = tfidf.transform(valid_df['processed_text'].apply(' '.join))

# Prepare the labels for train and valid dataset
train_labels = train_df['category']
valid_labels = valid_df['category']

# Logistic regression
logreg = LogisticRegression()
logreg.fit(training_vectors, train_labels)
logreg_predictions = logreg.predict(validation_vectors)

#random forest
random_forest = RandomForestClassifier()
random_forest.fit(training_vectors, train_labels)
rf_predictions = random_forest.predict(validation_vectors)

# analyxed the models
print("Logistic Regression Model:")
print(classification_report(valid_labels, logreg_predictions))
print("F1-score: ", f1_score(valid_labels, logreg_predictions, average='weighted'))

print("Random Forest Model:")
print(classification_report(valid_labels, rf_predictions))
print("F1-score: ", f1_score(valid_labels, rf_predictions, average='weighted'))

Observation: Logistic Regression is a simple, easy-to-understand linear model that works well for binary classification tasks, It can efficiently produce binary predictions by modelling the relationship between textual properties.    


OUTPUT:
When compared to the Logistic Regression model, the Random Forest model looks to be superior because it results in a higher weighted average F1-score.

In [ ]:
#!pip install tensorflowm

In [ ]:
#This code ws referenced from: https://www.tensorflow.org/tutorials/keras/save_and_load

#Here we will create a tensorflow model# 
import tensorflow as tf
#First we will assign the labels and target

# Separate the features (input data) and labels (target variable)
X = df['headline']
y = df['category']


# Convert the labels to numeric representation (0 and 1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create a tokenizer and fit on the training data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert the text data to sequences
X_train_sec = tokenizer.texts_to_sequences(X_train)
X_valid_sec = tokenizer.texts_to_sequences(X_valid)
X_test_sec = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to have the same length
max_sequence_length = max(len(sequence) for sequence in X_train_sec)
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_sec, maxlen=max_sequence_length)
X_valid_pad = tf.keras.preprocessing.sequence.pad_sequences(X_valid_sec, maxlen=max_sequence_length)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_sec, maxlen=max_sequence_length)

# Here we will define how will the model look
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# The below code will train the model
model.fit(X_train_pad, y_train, validation_data=(X_valid_pad, y_valid), epochs=10, batch_size=32)

# Lets now evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
print("Test sets Loss:", test_loss)
print("Test sets Accuracy:", test_accuracy)

#Save the model
model.save('book_classification_model.h5')

OBSERVATION: The model's training loss and accuracy gradually improve with each epoch during training, proving that it is learning from the data. The validation loss and accuracy both improve over the epochs, indicating that the model extends successfully to previously unseen data.

# Evaluation

In [ ]:
# Calculate the F1-score for the training set predictions
train_preds = model.predict(X_train_pad)
# Convert the predicted probabilities to binary predictions using a threshold of 0.5
binary_train_preds = [1 if prob >= 0.5 else 0 for prob in train_preds]
f1_train_score = f1_score(y_train, binary_train_preds)

# Calculate the F1-score for the validation set predictions
val_preds = model.predict(X_valid_pad)
binary_val_preds = [1 if prob >= 0.5 else 0 for prob in val_preds]
f1_val_score = f1_score(y_valid, binary_val_preds)

# Print the F1-scores for both training and validation sets
print("F1-score on train set:", f1_train_score)
print("F1-score on valid set:", f1_val_score)


On the training set, the F1-score is 1.0, indicating error-free performance. This indicates that the model has learned to categorize the training data effectively, achieving high precision and recall. The validation set's F1-score is 0.9079, which is slightly lower than the training set's F1-score. This difference indicates that the model generalizes well to new data but may not perform as well as it did on the training data.

In [ ]:
#The below code shows us how many records were predicted incorrectly when compared to its true labels

misclassified_samples = []
for i in range(len(y_valid)):
    if y_valid[i] != logreg_predictions[i] and y_valid[i] != rf_predictions[i]:
        misclassified_samples.append((X_valid_pad[i], y_valid[i], logreg_predictions[i], rf_predictions[i]))

# Analyze the misclassified samples
for sample in misclassified_samples:
    text, true_value, logreg_pred, rf_pred = sample
    print("Text values:", text)
    print("True Label values:", true_value)
    print("Logistic Regression Prediction values:", logreg_pred)
    print("Random Forest Prediction values:", rf_pred)
    print()

The code's output shows cases where both the Logistic Regression and Random Forest models predicted incorrectly on certain samples. By working more on them can result in good outcome 


In [ ]:
# Here we will get the number of misclassified predictions

incorrect_predictions = 0
for sample in misclassified_samples:
    true_value = sample[1]
    logreg_pred = sample[2]
    rf_pred = sample[3]
    
    if true_value != logreg_pred or true_value != rf_pred:
        incorrect_predictions += 1

print("Total incorrect predictions:", incorrect_predictions)


In [ ]:
#Loaded load and evaluate the model that we built

model_new = tf.keras.models.load_model("book_classification_model.h5")
model_new.summary()

The model has a total of 821,229 variables that can be trained. This indicates the model's predictive capability and the number of parameters learned during training.

Overall, the model appears to be an effective text categorization, and it has been loaded successfully for future evaluation and predictions.

In [ ]:
#got  loss and accuracy results
loss, acc = model_new.evaluate(X_test_pad,y_test, verbose = 2)

print("The accuracy of the model is: ",acc)
print("The loss of the model is: ",loss)

The loaded model's accuracy on the test set is around 96.18%, with a loss of approximately 0.1103. This shows the model's ability to make accurate predictions on new, text data.

In [ ]:
train_df
print()
valid_df

# MODELS 

In [ ]:
#Here we will tweak our classifiers that we used previously.

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

# Now you can use the PorterStemmer
stemmer = PorterStemmer()

# Now you can use the WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

#lemmatize the tokens
train_df['processed_text'] = train_df['processed_text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
valid_df['processed_text'] = valid_df['processed_text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])


#stem the tokens
train_df['processed_text'] = train_df['processed_text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
valid_df['processed_text'] = valid_df['processed_text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

In [ ]:
#convert the text data into numerical features using TF-IDF vector
tfidf = TfidfVectorizer(min_df=1) 
training_vectors = tfidf.fit_transform(train_df['processed_text'].apply(' '.join))
validation_vectors = tfidf.transform(valid_df['processed_text'].apply(' '.join))

#labels for training and validation
train_labels = train_df['category']
valid_labels = valid_df['category']
# train the Logistic Regression 
logreg_new = LogisticRegression(C=10, max_iter=1000, class_weight='balanced')
logreg_new.fit(training_vectors, train_labels)

#train the Random Forest 
random_forest_new = RandomForestClassifier(class_weight='balanced', n_estimators=500)
random_forest_new.fit(training_vectors, train_labels)

# Predict and evaluate the models on the validation set
logreg_predictions = logreg_new.predict(validation_vectors)
rf_predictions = random_forest_new.predict(validation_vectors)

# Print the results and evaluation metrics for Logistic Regression
print("Logistic Regression Results:")
print(classification_report(valid_labels, logreg_predictions))
print("F1-score Logistic Regression:", f1_score(valid_labels, logreg_predictions, average='weighted'))

# Print the results and evaluation metrics for Random Forest
print("Random Forest Results:")
print(classification_report(valid_labels, rf_predictions))
print("F1-score Random Forest:", f1_score(valid_labels, rf_predictions, average='weighted'))

to convert the text data in the training and validation datasets into numerical features, I use the TF-IDF vectorization technique. After that, I develop the labels for both datasets and start to train two separate models on the training data - the Logistic Regression model and the Random Forest model. The performance of these models on the validation dataset is next assessed by generating a classification report and calculating the F1 scores. It helps me to evaluate how well each model performs on validation data. Overall, I am preparing text, training, and testing machine learning models in order to better understand their capabilities on the given dataset.

In [ ]:
#Here we will perform the vectorization step using TF-IDF to convert the text data into numeric data

tfidf = TfidfVectorizer()
training_vectors = tfidf.fit_transform(train_df['processed_text'].apply(' '.join))
validation_vectors = tfidf.transform(valid_df['processed_text'].apply(' '.join))

#First we will prepare the labels to train
train_labels = train_df['category']
valid_labels = valid_df['category']
test_labels = test_df['category']


In [ ]:
#    logistic regression
logreg_new = LogisticRegression(C=10, max_iter=1000,class_weight='balanced')
logreg_new.fit(training_vectors, train_labels)

# Random Forest
random_forest_new = RandomForestClassifier(class_weight='balanced', n_estimators=500)
random_forest_new.fit(training_vectors, train_labels)

# Predict and evaluate the models on the validation set
logreg_predictions = logreg_new.predict(validation_vectors)
rf_predictions = random_forest_new.predict(validation_vectors)

# train the models
print("Logistic regression Model:")
print(classification_report(valid_labels, logreg_predictions))
print("F1-score: ", f1_score(valid_labels, logreg_predictions, average='weighted'))

print("Random Forest Model:")
print(classification_report(valid_labels, rf_predictions))
print("F1-score: ", f1_score(valid_labels, rf_predictions, average='weighted'))

I have the labels ready for training, validation, and test sets. Then, using the training data, I train a Logistic Regression model and a Random Forest model. I evaluate their performance using classification reports and F1-scores after generating predictions on the validation data. This procedure allows me to analyze the models' performance on unseen validation data and obtain information about their effectiveness.

In [ ]:
#Cross validation for recently created model.

# Assuming you have the train and validation sets as separate dataframes: train_df and valid_df
# Concatenate train and validation dataframes
merged_df = pd.concat([train_df, valid_df], ignore_index=True)

# Separate the merged dataframe into features and labels
text_data = merged_df['processed_text']
labels = merged_df['category']

joined_text = [' '.join(text) for text in text_data]

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from nltk.tokenize import word_tokenize

    # Initialize the TF-IDF vectorizer with a minimum document frequency
vectorizer = TfidfVectorizer(min_df=2)  # Adjust the value as per your data

# Transform the text data into numerical features
features_new = vectorizer.fit_transform(joined_text)

# Use SMOTE to balance the class distribution
oversampler = SMOTE(random_state=42)
features_new, labels = oversampler.fit_resample(features_new, labels)

# Initialize the models
logreg_new = LogisticRegression(class_weight='balanced', random_state=42)
random_forest_new = RandomForestClassifier(class_weight='balanced', random_state=42)

# Perform cross-validation for Logistic Regression
logreg_scores = cross_val_score(logreg_new, features_new, labels, cv=5, scoring='f1_macro', error_score='raise')
print("Logistic Regression Cross-Validation Scores:", logreg_scores)
print("Mean F1-score (Logistic Regression):", logreg_scores.mean())

# Perform cross-validation for Random Forest
rf_scores = cross_val_score(random_forest_new, features_new, labels, cv=5, scoring='f1_macro', error_score='raise')
print("Random Forest Cross-Validation Scores:", rf_scores)
print("Mean F1-score (Random Forest):", rf_scores.mean())


In this code, I used cross-validation to get a more accurate assessment of the model's performance and ability for generalization on fresh data. Also, by addressing class imbalance with SMOTE, I was able to significantly improve the random forest model's performance. The higher mean F1-score reflects this improvement, demonstrating better classification outcomes on unknown data. Overall, this approach proved a more robust and effective method for developing reliable classifiers, resulting in increased confidence in the model's prediction capabilities.

In [ ]:
test_df

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string

# Assuming you have defined test_df

# Preprocess the 'headline' column in test_df
def preprocess_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Remove punctuation from the words
    table = str.maketrans('', '', string.punctuation)
    words = [word.translate(table) for word in words]
    
    return words

# Apply the preprocessing function to the 'headline' column and store the result in 'processed_text' column
test_df['processed_text'] = test_df['headline'].apply(preprocess_text)

# Print the first few rows of test_df to see the results
print(test_df.head())

In [ ]:
# did TF-IDF vectorization to convert text data into numerical feature
tfidf = TfidfVectorizer(min_df=1)
training_vectors = tfidf.fit_transform(train_df['processed_text'].apply(' '.join))
test_vectors = tfidf.transform(test_df['processed_text'].apply(' '.join))

# label for training and test
train_labels = train_df['category']
test_labels = test_df['category'].astype('category').cat.codes

# Model: Train on training dataset only, test on test set
logreg_train = LogisticRegression(C=10, max_iter=1000, class_weight='balanced')
logreg_train.fit(training_vectors, train_labels)

# Predict and evaluate the model on the test set
logreg_train_predictions = logreg_train.predict(test_vectors)

# Evaluate the model
print("Logistic Regression Model (Training Data Only) - Test Set:")
print(classification_report(test_labels, logreg_train_predictions))
print("F1-score (Logistic Regression):", f1_score(test_labels, logreg_train_predictions, average='weighted'))


In comparison to previous codes, the current code is focused on precisely training a logistic regression model on the training dataset and assessing it on the test set.
in the current code, the logistic regression model achieves 92% accuracy on the test set, with a weighted F1-score of 0.92. This shows a strong ability to classify both good and negative events. In terms of evaluation, the logistic regression model classification report on the test set shows similar precision and recall values to earlier models.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

# Assuming you have defined train_df, valid_df, and test_df

# Concatenate train_df and valid_df into merged_df
merged_df = pd.concat([train_df, valid_df], ignore_index=True)

# Perform TF-IDF vectorization to convert text data into numerical features
tfidf = TfidfVectorizer(min_df=1)
training_vectors = tfidf.fit_transform(merged_df['processed_text'].apply(' '.join))
test_vectors = tfidf.transform(test_df['processed_text'].apply(' '.join))

# Prepare the labels for training and test
train_labels = merged_df['category']
test_labels = test_df['category'].astype('category').cat.codes

# Model: Train on combined training and validation datasets, test on test set
logreg_combined = LogisticRegression(C=10, max_iter=1000, class_weight='balanced')
logreg_combined.fit(training_vectors, train_labels)

# Predict and evaluate the model on the test set
logreg_combined_predictions = logreg_combined.predict(test_vectors)

# Evaluate the model
print("Logistic Regression Model (Combined Training and Validation Data) - Test Set:")
print(classification_report(test_labels, logreg_combined_predictions))
print("F1-score (Logistic Regression):", f1_score(test_labels, logreg_combined_predictions, average='weighted'))

Both models achieved identical precision and F1 scores, showing that the logistic regression model's performance did not change a lot when trained on the combined training and validation datasets. The F1 scores remained near, with the model utilizing merged data improving a bit.